# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 22:17:50.352000 180254 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:17:50.352000 180254 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:17:52] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=33681, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=132693810, constrained_

[2025-08-26 22:17:52] Using default HuggingFace chat template with detected content format: string


W0826 22:18:01.166000 181169 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:18:01.166000 181169 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 22:18:01.225000 181168 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:18:01.225000 181168 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:18:01] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-26 22:18:01] Init torch distributed begin.


...

Capturing batches (bs=2 avail_mem=32.42 GB):  50%|█████     | 2/4 [00:00<00:00, 12.90it/s]

Capturing batches (bs=1 avail_mem=32.40 GB): 100%|██████████| 4/4 [00:00<00:00, 15.02it/s]
[2025-08-26 22:18:23] Capture draft extend cuda graph end. Time elapsed: 1.88 s. mem usage=0.08 GB. avail mem=32.40 GB.
[2025-08-26 22:18:23] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=32.40 GB
[2025-08-26 22:18:23] INFO:     Started server process [180254]
[2025-08-26 22:18:23] INFO:     Waiting for application startup.
[2025-08-26 22:18:23] INFO:     Application startup complete.
[2025-08-26 22:18:23] INFO:     Uvicorn running on http://127.0.0.1:33681 (Press CTRL+C to quit)


[2025-08-26 22:18:23] INFO:     127.0.0.1:55882 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 22:18:24] INFO:     127.0.0.1:55884 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 22:18:24] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 22:18:25] INFO:     127.0.0.1:55900 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 22:18:25] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 22:18:29] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 22:18:29] INFO:     127.0.0.1:55904 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 22:18:36.377000 184065 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:18:36.377000 184065 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:18:39] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30468, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=315350763, constrained_js

[2025-08-26 22:18:39] Using default HuggingFace chat template with detected content format: string


W0826 22:18:48.642000 185255 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:18:48.642000 185255 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 22:18:48.652000 185254 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:18:48.652000 185254 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:18:49] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-26 22:18:49] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=36.27 GB): 100%|██████████| 4/4 [00:16<00:00,  4.09s/it]
[2025-08-26 22:20:21] Capture draft cuda graph end. Time elapsed: 17.08 s. mem usage=8.13 GB. avail mem=28.36 GB.
[2025-08-26 22:20:21] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=28.36 GB


Capturing batches (bs=1 avail_mem=28.26 GB): 100%|██████████| 4/4 [00:00<00:00, 24.64it/s]
[2025-08-26 22:20:22] Capture draft extend cuda graph end. Time elapsed: 1.03 s. mem usage=0.10 GB. avail mem=28.26 GB.
[2025-08-26 22:20:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=28.26 GB


[2025-08-26 22:20:22] INFO:     Started server process [184065]
[2025-08-26 22:20:22] INFO:     Waiting for application startup.
[2025-08-26 22:20:22] INFO:     Application startup complete.
[2025-08-26 22:20:22] INFO:     Uvicorn running on http://127.0.0.1:30468 (Press CTRL+C to quit)


[2025-08-26 22:20:23] INFO:     127.0.0.1:42598 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-26 22:20:23] INFO:     127.0.0.1:42600 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 22:20:23] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 22:20:25] INFO:     127.0.0.1:42614 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 22:20:25] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 22:20:28] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 22:20:29] INFO:     127.0.0.1:42624 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 22:20:36.229000 194232 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:20:36.229000 194232 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:20:37] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34229, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=112180410,

[2025-08-26 22:20:38] Using default HuggingFace chat template with detected content format: string


W0826 22:20:45.474000 194468 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:20:45.474000 194468 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 22:20:45.543000 194467 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:20:45.543000 194467 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:20:46] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=41.21 GB): 100%|██████████| 4/4 [00:06<00:00,  1.74s/it]
[2025-08-26 22:21:14] Capture draft cuda graph end. Time elapsed: 7.56 s. mem usage=1.11 GB. avail mem=41.17 GB.
[2025-08-26 22:21:14] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=41.17 GB


Capturing batches (bs=1 avail_mem=41.07 GB): 100%|██████████| 4/4 [00:00<00:00, 29.57it/s]
[2025-08-26 22:21:14] Capture draft extend cuda graph end. Time elapsed: 0.80 s. mem usage=0.10 GB. avail mem=41.07 GB.
[2025-08-26 22:21:14] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=41.07 GB


[2025-08-26 22:21:15] INFO:     Started server process [194232]
[2025-08-26 22:21:15] INFO:     Waiting for application startup.
[2025-08-26 22:21:15] INFO:     Application startup complete.
[2025-08-26 22:21:15] INFO:     Uvicorn running on http://127.0.0.1:34229 (Press CTRL+C to quit)


[2025-08-26 22:21:16] INFO:     127.0.0.1:52610 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-26 22:21:16] INFO:     127.0.0.1:52622 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 22:21:16] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 22:21:17] INFO:     127.0.0.1:52634 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 22:21:17] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 22:21:21] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 22:21:21] INFO:     127.0.0.1:57618 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 22:21:29.237000 197118 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:21:29.237000 197118 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:21:30] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=37258, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=165232707, const

[2025-08-26 22:21:30] Using default HuggingFace chat template with detected content format: string


W0826 22:21:37.836000 198015 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:21:37.836000 198015 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 22:21:37.843000 198016 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:21:37.843000 198016 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:21:38] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=57.73 GB): 100%|██████████| 4/4 [00:05<00:00,  1.31s/it]
[2025-08-26 22:22:04] Capture draft cuda graph end. Time elapsed: 5.74 s. mem usage=4.34 GB. avail mem=54.09 GB.
[2025-08-26 22:22:04] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=54.09 GB


Capturing batches (bs=1 avail_mem=53.19 GB): 100%|██████████| 4/4 [00:00<00:00, 75.10it/s]
[2025-08-26 22:22:05] Capture draft extend cuda graph end. Time elapsed: 0.72 s. mem usage=1.00 GB. avail mem=53.09 GB.
[2025-08-26 22:22:05] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=53.09 GB


[2025-08-26 22:22:05] INFO:     Started server process [197118]
[2025-08-26 22:22:05] INFO:     Waiting for application startup.
[2025-08-26 22:22:05] INFO:     Application startup complete.
[2025-08-26 22:22:05] INFO:     Uvicorn running on http://127.0.0.1:37258 (Press CTRL+C to quit)
[2025-08-26 22:22:05] INFO:     127.0.0.1:38020 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-26 22:22:06] INFO:     127.0.0.1:38032 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 22:22:06] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-26 22:22:07] INFO:     127.0.0.1:38040 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 22:22:07] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-26 22:22:10] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-26 22:22:11] INFO:     127.0.0.1:38048 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0826 22:22:18.673000 201548 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:22:18.673000 201548 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


A new version of the following files was downloaded from https://huggingface.co/XiaomiMiMo/MiMo-7B-RL:
- configuration_mimo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
- configuration_mimo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
[2025-08-26 22:22:19] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=39679, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_runn

[2025-08-26 22:22:20] Using default HuggingFace chat template with detected content format: string


W0826 22:22:27.159000 202281 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:22:27.159000 202281 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0826 22:22:27.161000 202282 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0826 22:22:27.161000 202282 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-26 22:22:28] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-26 22:22:28] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=36.49 GB):  75%|███████▌  | 3/4 [00:01<00:00,  2.18it/s]

Capturing batches (bs=1 avail_mem=36.49 GB): 100%|██████████| 4/4 [00:02<00:00,  1.72it/s]
[2025-08-26 22:23:21] Capture draft cuda graph end. Time elapsed: 3.09 s. mem usage=-1.66 GB. avail mem=41.48 GB.
[2025-08-26 22:23:21] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=41.48 GB


Capturing batches (bs=1 avail_mem=41.39 GB): 100%|██████████| 4/4 [00:00<00:00, 25.61it/s]
[2025-08-26 22:23:22] Capture draft extend cuda graph end. Time elapsed: 0.72 s. mem usage=0.09 GB. avail mem=41.39 GB.
[2025-08-26 22:23:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=41.39 GB


[2025-08-26 22:23:22] INFO:     Started server process [201548]
[2025-08-26 22:23:22] INFO:     Waiting for application startup.
[2025-08-26 22:23:22] INFO:     Application startup complete.
[2025-08-26 22:23:22] INFO:     Uvicorn running on http://0.0.0.0:39679 (Press CTRL+C to quit)


[2025-08-26 22:23:23] INFO:     127.0.0.1:55742 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-26 22:23:23] INFO:     127.0.0.1:55752 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-26 22:23:23] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-26 22:24:10] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-26 22:24:11] INFO:     127.0.0.1:55760 - "POST /generate HTTP/1.1" 200 OK
[2025-08-26 22:24:11] The server is fired up and ready to roll!


[2025-08-26 22:24:12] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.63, cuda graph: True, gen throughput (token/s): 1.46, #queue-req: 0, 


[2025-08-26 22:24:12] Decode batch. #running-req: 1, #token: 160, token usage: 0.01, accept len: 1.70, cuda graph: True, gen throughput (token/s): 176.19, #queue-req: 0, 


[2025-08-26 22:24:12] Decode batch. #running-req: 1, #token: 225, token usage: 0.01, accept len: 1.62, cuda graph: True, gen throughput (token/s): 160.94, #queue-req: 0, 


...

[2025-08-26 22:24:13] Decode batch. #running-req: 1, #token: 363, token usage: 0.02, accept len: 1.80, cuda graph: True, gen throughput (token/s): 160.91, #queue-req: 0, 


[2025-08-26 22:24:14] Decode batch. #running-req: 1, #token: 421, token usage: 0.02, accept len: 1.45, cuda graph: True, gen throughput (token/s): 146.37, #queue-req: 0, 


[2025-08-26 22:24:14] Decode batch. #running-req: 1, #token: 487, token usage: 0.02, accept len: 1.65, cuda graph: True, gen throughput (token/s): 168.48, #queue-req: 0, 


[2025-08-26 22:24:14] Decode batch. #running-req: 1, #token: 556, token usage: 0.03, accept len: 1.73, cuda graph: True, gen throughput (token/s): 177.05, #queue-req: 0, 


[2025-08-26 22:24:15] Decode batch. #running-req: 1, #token: 628, token usage: 0.03, accept len: 1.80, cuda graph: True, gen throughput (token/s): 183.20, #queue-req: 0, 
[2025-08-26 22:24:15] INFO:     127.0.0.1:55772 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).